<a href="https://colab.research.google.com/github/AditisGit/FraudCreditCardTransactionDetector/blob/main/Credit_Card_Fraud_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CREDITCARD FRAUD DETECTION USING LINEAR REGRESSION

---



# Importing dataset from Kaggle:

---



In [ ]:
!pip install kaggle
from google.colab import files 
loader = files.upload()

In [ ]:
!mkdir -p ~/.kaggle 
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d mlg-ulb/creditcardfraud

In [ ]:
!pip install zip_files 

In [ ]:
from zipfile import ZipFile
creditcard_data = 'creditcardfraud.zip'
with ZipFile(creditcard_data,'r') as zip:
  zip.extractall()

#Importing the required Python Libraries:

---



In [ ]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 
import tensorflow as tf 
from keras.layers import Dropout, Dense, Flatten
from keras.models import Sequential
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
%matplotlib inline

# Reading and making inferences from the dataset: 

---




In [ ]:
credit_df = pd.read_csv('creditcard.csv')

In [ ]:
credit_df.head(5)

In [ ]:
credit_df.tail(5)

In [ ]:
credit_df.info()

In [ ]:
credit_df.describe()

In [ ]:
credit_df.isna().any()

# Visualisation of data

In [ ]:
sns.catplot(data=credit_df, kind="bar",x="Class", y="Amount", palette="dark", alpha=.6, height=6)

In [ ]:
sns.scatterplot(data=credit_df, x='Class', y='Amount', palette='dark')

In [ ]:
X = credit_df[['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28']]
y = credit_df[['Class']]
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state=0)

In [ ]:
scaler = StandardScaler()
credit_df['Normalized Amount'] = scaler.fit_transform(credit_df['Amount'].values.reshape(-1,1))

In [ ]:
credit_df = credit_df.drop(['Normalized Amount'], axis=1)

# Training the Model:

---
The model used for this project contains one input layer,one output layer and two hidden layers. The ReLU activation function is used for the first and hidden layers and the Output layer uses a Sigmoid activation function.
The model is trained over five epochs and uses the adam optimizer and binary crossentropy function for loss to calculate gradient descent while compiling the model.
The model is trained by splitting the dataset into Train and Test sets. The Training set contains 1,99,364 transactions and the test set contains 85,443 transactions. However, the dataset is highly unbalanced and contains only 492 fraudulent transactions.



In [ ]:
model = Sequential([
                    Dense(units=20, input_dim=28, activation='relu'),
                    Dense(units=15, activation='relu'),
                    Dropout(0.2),
                    Dense(units=10, activation='relu'),
                    Dense(units=1, activation='sigmoid')

])

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
r = model.fit(X_train, y_train, validation_data=(X_test,y_test), epochs=5)

# Visualising the Loss and Accuracy of predictions

---
A plot of the loss versus validation loss and accuracy versus calidation accuracy allows us to check if the model is being over-fitted or underfitted.
The model trained here shows low loss and high accuracy and seems to be doing fine.



In [ ]:
plt.plot(r.history['loss'],label='Loss')
plt.plot(r.history['val_loss'],label='Val_loss')
plt.legend()

In [ ]:
plt.plot(r.history['accuracy'],label='accuracy')
plt.plot(r.history['val_accuracy'],label='Val_accuracy')
plt.legend()

In [ ]:
print(model.evaluate(X_test, y_test))
p_test = model.predict(X_test).argmax(axis=1)

The model shows a prediction accuracy of 0.9994

# Evaluation of the model using a confuison matrix

---
A plot of the confusion matrix is a visualisation of the predictive analysis and enables us to understand its performance better.


In [ ]:
print(classification_report(y_test, p_test))